## **Lectura 4: Marco general para Formulación implícita**

<H3>Generalizar el problema de la dieta</H3>

Recordemos que el objetivo del problema de la dieta es encontrar una combinación de alimentos que satisfaga algunos requisitos de nutrientes. Es sencillo escribir un modelo para un pequeño conjunto de entradas como el siguiente.

Unidades de nutrientes y costo por onza de alimento:
| Food type | Iron | Calcium | Cost |
|-----------|------|---------|------|
| 1         | 2    | 0       | 20   |
| 2         | 0    | 1       | 10   |
| 3         | 3    | 2       | 31   |
| 4         | 1    | 2       | 11   |
| 5         | 2    | 1       | 12   |


Necesidades de nutrientes: 21 unidades de hierro y 12 unidades de calcio

Podemos definir simplemente nuestras variables de decisión $x_1,\ldots,x_5$ como el número de onzas a consumir de cada tipo de alimento, y el programa lineal resultante es simplemente:

$$ min_x \quad  20 x_1 + 10 x_2 + 31 x_3 + 11 x_4 + 12 x_5 $$

$$text{s.t.} \quad  2 x_1 + 0 x_2 + 3 x_3 + 1 x_4 + 2 x_5 \ge 21 $$

$$ 0 x_1 + 1 x_2 + 2 x_3 + 2 x_4 + 1 x_5 \ge 12 $$

$$ x_1,\ldots,x_5 \ge 0 $$

Escribir este pequeño LP es un ejercicio útil, pero necesitaremos escribir una versión más general del modelo si alguna vez queremos escribir un programa que pueda resolver cualquier instancia del problema de la dieta.

<H3>Conjuntos e índices</H3>

Un LP consta de variables de decisión, restricciones y un objetivo, todos los cuales tendremos que definir, pero ninguno de los cuales podemos definir hasta que creemos una notación para los distintos conjuntos del problema. Para el problema de la dieta, las entidades relevantes son los nutrientes y los tipos de alimentos. Empezaremos definiendo los siguientes conjuntos:

* $i \in I$: nutrientes
* $j \in J$: tipos de alimentos

La convención que preferimos es utilizar una letra mayúscula para denotar el conjunto completo, y una letra minúscula para denotar un elemento de ese conjunto. El símbolo $\in$ puede leerse como ``en'', por lo que $i \in I$ indica que $i$ es un nutriente concreto que está en $I$, el conjunto completo de nutrientes.

Definir los conjuntos relevantes suele ser el primer paso en la modelización.

<H3>Datos</H3>

Una vez que hemos definido nuestros conjuntos, podemos tomar nuestros datos de entrada y escribirlos de una manera más general. Dado que las LP pueden tener cualquier combinación de restricciones $\ge$, $\le$ y $=$, podemos generalizar los requisitos de nutrientes para incluir un límite inferior y superior para cada nutriente.

* $c_j$: coste por onza del tipo de alimento $j$
* $a_{ij}$: cantidad de nutriente $i$ por onza de alimento del tipo $j$
* $l_i, u_i$: necesidades diarias mínimas y máximas del nutriente $i$

<H3>Variables de decisión</H3>

* $x_j$: el número de onzas a consumir del tipo de alimento $j$.

Con las variables de decisión y los datos escritos de forma genérica, podemos escribir expresiones para el coste total, y para la cantidad de cada nutriente en nuestra dieta.

<H3>Objetivo</H3>

El coste total puede obtenerse multiplicando el número de onzas consumidas de un tipo de alimento, $x_j$, por el coste por onza de ese tipo de alimento, $c_j$, y sumando todos los tipos de alimentos $j \in J$. Utilizaremos la notación $\sum$ para denotar sumas, poniendo el conjunto sobre el que estamos sumando bajo $\sum$. Por lo tanto, el objetivo se puede escribir como $\sum_{j \in J} c_j x_j$.

<H3>Restricciones</H3>

Para escribir las restricciones que imponen límites al consumo de nutrientes, necesitaremos escribir una expresión para la cantidad de cada nutriente que consumimos. 

Fijemos el nutriente $i$. La contribución del tipo de alimento $j$ al nutriente $i$ será el producto de la cantidad por onza del nutriente $i$, $a_{ij}$, por el número de onzas consumidas, $x_j$. Vamos a sumar este producto sobre todos los tipos de alimentos $j \in J$, de nuevo utilizando $\sum$ notación. La expresión resultante será $\sum_{j \in J} a_{ij}x_j$.

Esta expresión es válida para cualquier nutriente $i \in I$. Esto nos da todo lo que necesitamos para formular el problema de la dieta como un LP.

<H3>Formulation</H3>

$$ \min_x \quad  \sum_{j \in J} c_j x_j $$ 
$$ \text{s.t.} \quad  l_i \le \sum_{j \in J} a_{ij} x_j \le u_i, \quad i \in I $$ 
$$  x_j \ge 0, \quad j \in J.$$ 

Ahora vamos a implementar el método solve_diet_problem. Tenemos una restricción que pone un límite inferior y superior en una expresión lineal. Encontraremos el método Model.addRange útil para esto, y preferimos usarlo en lugar de hacer dos llamadas a Model.addConstr.

In [13]:
import gurobipy as grb
from gurobipy import GRB

grb.Model.addRange?

Signature:      grb.Model.addRange(self, expr, lower, upper, name='')
Call signature: grb.Model.addRange(*args, **kwargs)
Type:           cython_function_or_method
String form:    <cyfunction Model.addRange at 0x000001F8AD7B4930>
Docstring:     
ROUTINE:
  addRange(expr, lhs, rhs, name)

PURPOSE:
  Add a range constraint to the model.

ARGUMENTS:
  expr (Var, or LinExpr): Linear expression being constrained
  lower (float): Lower bound on linear expression
  upper (float): Upper bound on linear expression
  name (string): Constraint name (default is no name)

RETURN VALUE:
  The created Constr object.

EXAMPLE:
  c = model.addRange(x + y, 1.0, 2.0)

In [14]:
def solve_diet_problem(nutrient_densities, costs, nutrient_requirements):
    """
    Esta función está diseñada para encontrar la dieta más barata que satisface ciertos requisitos nutricionales.

    Entradas:
  
    :param nutrient_densities: Diccionario que mapea cada tipo de alimento y nutriente a su densidad nutricional.
    :param costs: Diccionario que mapea cada tipo de alimento a su costo.
    :param nutrient_requirements: Diccionario que mapea cada nutriente a un rango (min, max) de requerimiento.

    Salida:
    :return: Diccionario que mapea cada tipo de alimento a la cantidad óptima a consumir.
    :raises Exception: Se lanza una excepción si el modelo es inviable.
    """

    # Creamos un modelo matemático en Gurobi. Piensa en esto como un lienzo vacío donde añadiremos nuestras decisiones y restricciones.
    m = grb.Model()
    
    # Para cada tipo de alimento, decidimos cuánto consumir. Estas son nuestras "variables de decisión".
    # También decimos a Gurobi que queremos minimizar el costo total de los alimentos que consumimos.
    ounces_consumed = {}
    for food_type, cost in costs.items():
        var = m.addVar(obj=cost, name='ounces_consumed.' + str(food_type))
        ounces_consumed[food_type] = var

    # Hacemos que Gurobi sepa de las nuevas variables.
    m.update()

    # Ahora, añadimos las restricciones de nutrientes.
    # Para cada nutriente, calculamos cuánto consumimos en total basado en nuestras decisiones de alimentos.
    # Luego, nos aseguramos de que esta cantidad esté entre los valores mínimos y máximos que queremos.
    for nutrient, (min_requirement, max_requirement) in nutrient_requirements.items():
        total_nutrient_consumed = sum(nutrient_densities[food_type, nutrient] * ounces_consumed[food_type] for food_type in costs.keys())
        
        m.addRange(total_nutrient_consumed, min_requirement, max_requirement, 'nutrient.' + str(nutrient))

    # ¡Listo! Ahora le decimos a Gurobi que encuentre la mejor solución.
    m.optimize()
    
    # Si Gurobi encontró una solución que cumple con nuestros requisitos nutricionales y minimiza el costo, nos dice cuánto de cada alimento consumir.
    # Si no, nos avisa que no fue posible encontrar una dieta que cumpla con las condiciones.
    if m.status == GRB.OPTIMAL:
        return {food_type: var.X for food_type, var in ounces_consumed.items()}
    
    raise Exception("Modelo infactible, no fue posible encontrar una dieta que cumpla con los requisitos.")


Probemos nuestra función con el problema de la dieta generalizado. Para ello, primero debemos cargar los datos del problema.

**Definir instancias**

In [15]:
# Instancia 1
nutrient_densities_1 = {
    ('apple', 'vitamin_c'): 4.6,
    ('banana', 'vitamin_c'): 8.7,
    ('carrot', 'vitamin_c'): 2.8
}

costs_1 = {
    'apple': 0.5,
    'banana': 0.4,
    'carrot': 0.3
}

nutrient_requirements_1 = {
    'vitamin_c': (10, 50)  # Requerimos entre 10 y 50 unidades de vitamina C
}

# Instancia 2
nutrient_densities_2 = {
    ('apple', 'vitamin_c'): 4.6,
    ('banana', 'vitamin_c'): 8.2,  # Cambiamos ligeramente la densidad nutricional
    ('carrot', 'vitamin_c'): 3.1  # Cambiamos ligeramente la densidad nutricional
}

costs_2 = {
    'apple': 0.55,  # Cambiamos ligeramente el costo
    'banana': 0.38,  # Cambiamos ligeramente el costo
    'carrot': 0.32  # Cambiamos ligeramente el costo
}

nutrient_requirements_2 = {
    'vitamin_c': (12, 45)  # Cambiamos ligeramente los requerimientos
}


In [16]:
result_1 = solve_diet_problem(nutrient_densities_1, costs_1, nutrient_requirements_1)
result_2 = solve_diet_problem(nutrient_densities_2, costs_2, nutrient_requirements_2)

print("Resultados para la Instancia 1:", result_1)
print("Resultados para la Instancia 2:", result_2)


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xa75ab635
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [3e-01, 5e-01]
  Bounds range     [4e+01, 4e+01]
  RHS range        [5e+01, 5e+01]
Presolve removed 1 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5977011e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.597701149e-01
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads


## Ejercicio

Prueba a crear una función para automatizar la generación de instancias aleatorias de problemas de dieta. Esto es especialmente útil si estás tratando de probar la robustez o la eficiencia de tu función con diferentes conjuntos de datos.